# 从零开始实现

In [13]:
import random
import torch
from d2l import torch as d2l

In [14]:
#生成合成数据集
def synthetic_data(w,b,num):
    X=torch.normal(0,1,(num,len(w)))
    y=torch.matmul(X,w)+b
    y+=torch.normal(0,0.01,y.shape)#加入服从均值为0，标准差为0.01的正态分布的噪声
    return X,y.reshape((-1,1))



true_w = torch.tensor([2,-3.4]).float()# 真实值
true_b=4.2
features, labels= synthetic_data(true_w,true_b,1000)

In [15]:
#打乱数据集样本，并抽取小批量样本。
def data_iter(batch_size, features, labels):
    num=len(features)
    indices=list(range(num))
    random.shuffle(indices)#打乱下标顺序
    for i in range(0,num,batch_size):
        batch_indices=torch.tensor(indices[i:min(i+batch_size,num)])#防止越界       
        yield features[batch_indices],labels[batch_indices]

In [16]:
#初始化参数模型。
w=torch.normal(0,0.01,size=(2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)  #需要计算梯度来更新参数
batch_size=10

## 定义模型

In [17]:
def linreg(X,w,b):
    return torch.matmul(X,w)+b


## 定义平方损失函数


In [18]:
def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2/2
    

## 定义优化算法(小批量梯度下降)

In [19]:
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr* param.grad/batch_size
            param.grad.zero_()

In [22]:
lr=1
num_epochs=3
net=linreg
loss=squared_loss

for epoch in range(num_epochs):
    for X,y in data_iter(batch_size, features, labels):
        l=loss(net(X,w,b),y)
        l.sum().backward()  #将l转化为标量
        sgd([w,b],lr,batch_size) #更新参数
    with torch.no_grad():
        train_l=loss(net(features,w,b),labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.000110
epoch 2, loss 0.000066
epoch 3, loss 0.000058


# 使用框架实现

In [26]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)


In [29]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset= data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)
batch_size=10
data_iter = load_array((features,labels),batch_size)

In [44]:
from torch import nn
net=nn.Sequential(nn.Linear(2,1))#输入特征形状，输出特征形状
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [45]:
loss=nn.MSELoss()
trainer=torch.optim.SGD(net.parameters(),lr=0.03)

In [46]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l=loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000229
epoch 2, loss 0.000105
epoch 3, loss 0.000104
